In [1]:
import pandas as pd 
import pickle
import warnings
warnings.filterwarnings('ignore')
import numpy as np

from sklearn.metrics import classification_report
from sklearn import preprocessing
import torch
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.autograd import Variable

In [2]:
y_train = pd.read_csv("/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/lexical/text_feature_extraction/Emotion Roberta/embeddings/train_labels.csv")
y_test = pd.read_csv("/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/lexical/text_feature_extraction/Emotion Roberta/embeddings/test_labels.csv")


In [3]:
with open('/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/lexical/text_feature_extraction/Emotion Roberta/embeddings/train_emoberta_embeddings_target_context.pkl', 'rb') as f:
    x_train = pickle.load(f, encoding='latin1')

with open('/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/lexical/text_feature_extraction/Emotion Roberta/embeddings/test_emoberta_embeddings_target_context.pkl', 'rb') as f:
    x_test = pickle.load(f, encoding='latin1')

In [4]:
x_train_vals = []
for sample in x_train["embeddings"]:
    x_train_vals.append(torch.stack(sample))
    
x_test_vals = []
for sample in x_test["embeddings"]:
    x_test_vals.append(torch.stack(sample))

x_train_df = pd.DataFrame({'embeddings':x_train_vals})
x_test_df = pd.DataFrame({'embeddings':x_test_vals})

x_train_df["sarcasm"] = y_train["sarcasm"]
x_test_df["sarcasm"] = y_test["sarcasm"]
x_train_df["sarcasm"] = x_train_df["sarcasm"].astype('int').to_numpy()
x_test_df["sarcasm"] = x_test_df["sarcasm"].astype('int').to_numpy()

In [15]:
class RNNTensorDataset(Dataset):
    def __init__(self, dataframe, speaker):
        self.data = dataframe
        self.speaker = speaker

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        if self.speaker:
            features = self.data.loc[index, 'embeddings']
            a=np.empty((512,1))
            a.fill(self.data.loc[index, 'speaker_encode'])
            final_features = np.hstack((features, a))
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(final_features).float(), label
        else:
            features = self.data.loc[index, 'embeddings']
            label = self.data.loc[index, 'sarcasm']
            return features.float(), label
    
    def __getindexlist__(self):
        return list(self.data.index.values)
    
class RNNetSD(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(RNNetSD, self).__init__()
        self.hidden_dim = hidden_dim
        self.layers = layers
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True, nonlinearity="relu")
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = Variable(torch.zeros(self.layers, x.size(0), self.hidden_dim))
        out, hn = self.rnn(x, h0)
        out = F.softmax(self.fc(out[:, -1, :]))
        return out
    
class RNNetSID(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(RNNetSID, self).__init__()
        self.hidden_dim = hidden_dim
        self.layers = layers
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True, nonlinearity="relu")
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = Variable(torch.zeros(self.layers, x.size(0), self.hidden_dim))
        out, hn = self.rnn(x, h0)
        out = F.softmax(self.fc(out[:, -1, :]))
        return out
    
def evaluateRNN(rnn, review, size):
    output = rnn(review)
    return output

def categoryFromOutput(output):
    top_n, top_i = torch.max(output,dim=1)
    return top_i

def test_accuracy(rnn, loader, size):
    actuals = []
    predictions = []
    for data, target in loader:
        output = evaluateRNN(rnn, data, size)
        prediction_index = categoryFromOutput(output)
        predictions = prediction_index.tolist()
        actuals = target.tolist()
    return predictions, actuals

In [16]:
EMBEDDING_DIM_sid = 768
EMBEDDING_DIM_sd = 769
HIDDEN_DIM = 50
OUTPUT_DIM = 2
layers = 2
criterion = nn.NLLLoss()

### Speaker Independent and Context Dependent

In [17]:
rnn_train_tensor = RNNTensorDataset(x_train_df[['embeddings', 'sarcasm']], False)
rnn_test_tensor = RNNTensorDataset(x_test_df[['embeddings', 'sarcasm']], False)

num_of_workers = 0
batch_size = 44
valid_size = 0.1

train_indices = list(range(len(rnn_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(rnn_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    rnn_train_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(train_indices)
)

test_loader = torch.utils.data.DataLoader(
    rnn_test_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(test_indices)
)

test_loader_epoch = torch.utils.data.DataLoader(
    rnn_test_tensor, batch_size=rnn_test_tensor.__len__())

In [18]:
rnn = RNNetSID(EMBEDDING_DIM_sid, HIDDEN_DIM, OUTPUT_DIM, layers)
print(rnn)

RNNetSID(
  (rnn): RNN(768, 50, num_layers=2, batch_first=True)
  (fc): Linear(in_features=50, out_features=2, bias=True)
)


In [19]:
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001)

n_epochs = 11
    
test_min_loss = np.inf

for epoch in range(n_epochs):
    torch.manual_seed(42)
    train_loss = 0.0
    test_loss = 0.0
    rnn.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = rnn(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    rnn.eval()
    for data, target in test_loader:
        if data.shape[1] < 44:
            continue
        output = rnn(data)
        loss = criterion(output, target)
        test_loss += loss.item()*data.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    if(epoch%1 == 0):
        print("Epoch: " + str(epoch))
        predictions, actuals = test_accuracy(rnn, test_loader_epoch, rnn_test_tensor.__len__())
        print(classification_report(actuals, predictions, digits=4))

Epoch: 0
              precision    recall  f1-score   support

           0     0.5021    1.0000    0.6685       121
           1     0.0000    0.0000    0.0000       120

    accuracy                         0.5021       241
   macro avg     0.2510    0.5000    0.3343       241
weighted avg     0.2521    0.5021    0.3356       241

Epoch: 1
              precision    recall  f1-score   support

           0     0.5779    0.7355    0.6473       121
           1     0.6322    0.4583    0.5314       120

    accuracy                         0.5975       241
   macro avg     0.6051    0.5969    0.5893       241
weighted avg     0.6049    0.5975    0.5896       241

Epoch: 2
              precision    recall  f1-score   support

           0     0.5303    0.8678    0.6583       121
           1     0.6279    0.2250    0.3313       120

    accuracy                         0.5477       241
   macro avg     0.5791    0.5464    0.4948       241
weighted avg     0.5789    0.5477    0.4955    

### Speaker Dependent and Context Dependent

In [20]:
x_train_speakers = pd.read_csv("/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/lexical/text_feature_extraction/Emotion Roberta/embeddings/train_data.csv")
x_test_speakers = pd.read_csv("/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/lexical/text_feature_extraction/Emotion Roberta/embeddings/test_data.csv")
x_train_speakers

,target_,target_context,speaker
0,<s> i have been told it is a good way to move...,<s> i have been told it is a good way to move...,25
1,"<s> yeah, sure. you slept with your husband. ...","<s> yeah, sure. you slept with your husband. ...",1
2,<s> when are you coming home? </s>,<s> when are you coming home? okay. alright....,16
3,<s> riveting. </s>,<s> riveting. bingo. then i lifted the cushi...,0
4,"<s> no, this is just part of a daredevil game...","<s> no, this is just part of a daredevil game...",2
...,...,...,...
956,"<s> oh, that is sweet, but today is all about...","<s> oh, that is sweet, but today is all about...",7
957,<s> if you want to put a label on it. </s>,<s> if you want to put a label on it. you me...,24
958,<s> that you are an alcoholic? </s>,<s> that you are an alcoholic? i realized so...,3
959,<s> all i see is a yellow smudge. </s>,<s> all i see is a yellow smudge. now go bac...,15


In [21]:
x_train_df["speaker_encode"] = x_train_speakers["speaker"]
x_test_df["speaker_encode"] = x_test_speakers["speaker"]
x_train_df

,embeddings,sarcasm,speaker_encode
0,"[[tensor(-0.2603), tensor(-0.0016), tensor(-0....",0,25
1,"[[tensor(-0.1349), tensor(-0.1149), tensor(-0....",0,1
2,"[[tensor(-0.2174), tensor(-0.2594), tensor(-0....",0,16
3,"[[tensor(0.0198), tensor(-0.0934), tensor(-0.1...",1,0
4,"[[tensor(-0.1553), tensor(-0.2999), tensor(-0....",1,2
...,...,...,...
956,"[[tensor(-0.1470), tensor(0.1035), tensor(0.01...",0,7
957,"[[tensor(0.0305), tensor(0.0200), tensor(-0.23...",1,24
958,"[[tensor(0.0984), tensor(-0.2973), tensor(-0.1...",1,3
959,"[[tensor(-0.2136), tensor(-0.2924), tensor(-0....",0,15


In [22]:
rnn_train_tensor = RNNTensorDataset(x_train_df[['embeddings', 'sarcasm', 'speaker_encode']], True)
rnn_test_tensor = RNNTensorDataset(x_test_df[['embeddings', 'sarcasm', 'speaker_encode']], True)

num_of_workers = 0
batch_size = 44
valid_size = 0.1

train_indices = list(range(len(rnn_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(rnn_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    rnn_train_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(train_indices)
)

test_loader = torch.utils.data.DataLoader(
    rnn_test_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(test_indices)
)

test_loader_epoch = torch.utils.data.DataLoader(
    rnn_test_tensor, batch_size=rnn_test_tensor.__len__())

In [23]:
rnn = RNNetSD(EMBEDDING_DIM_sd, HIDDEN_DIM, OUTPUT_DIM, layers)
print(rnn)

optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001)

n_epochs = 11
    
test_min_loss = np.inf

for epoch in range(n_epochs):
    torch.manual_seed(42)
    train_loss = 0.0
    test_loss = 0.0
    rnn.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = rnn(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    rnn.eval()
    for data, target in test_loader:
        if data.shape[1] < 44:
            continue
        output = rnn(data)
        loss = criterion(output, target)
        test_loss += loss.item()*data.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    if(epoch%1 == 0):
        print("Epoch: " + str(epoch))
        predictions, actuals = test_accuracy(rnn, test_loader_epoch, rnn_test_tensor.__len__())
        print(classification_report(actuals, predictions, digits=4))

RNNetSD(
  (rnn): RNN(769, 50, num_layers=2, batch_first=True)
  (fc): Linear(in_features=50, out_features=2, bias=True)
)
Epoch: 0
              precision    recall  f1-score   support

           0     0.5765    0.8099    0.6735       121
           1     0.6761    0.4000    0.5026       120

    accuracy                         0.6058       241
   macro avg     0.6263    0.6050    0.5881       241
weighted avg     0.6261    0.6058    0.5884       241

Epoch: 1
              precision    recall  f1-score   support

           0     0.6296    0.5620    0.5939       121
           1     0.6015    0.6667    0.6324       120

    accuracy                         0.6141       241
   macro avg     0.6156    0.6143    0.6131       241
weighted avg     0.6156    0.6141    0.6131       241

Epoch: 2
              precision    recall  f1-score   support

           0     0.6562    0.5207    0.5806       121
           1     0.6000    0.7250    0.6566       120

    accuracy                    